In [1]:
import numpy as np
import pandas as pds
import pandas as pd
import sqlite3

In [2]:
lights = pd.read_csv('data/Lights.csv')
collision = pd.read_csv('data/NYPD_Motor_Vehicle_Collisions.csv')

/home/lee/anaconda3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (17,18,42,43,44,45,46,47,49) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
lights.shape, collision.shape

((791246, 53), (994499, 29))

In [4]:
lights = lights.dropna(subset=['Latitude', 'Longitude'])
collision = collision.dropna(subset=['LATITUDE', 'LONGITUDE'])

# Remove unusable data from Lights

In [5]:
lights = lights[lights['Complaint Type'] == 'Street Light Condition']

In [6]:
descriptors_useful = ['Street Light Out', 
                      'Fixture/Luminaire Hanging', 
                      'Lamppost Knocked Down', 
                      'Fixture/Luminaire Damaged', 
                      'Lamppost Missing', 
                      'Lamppost Damaged',
                      'Street Light Lamp Missing', 
                      'Fixture/Luminaire Missing', 
                      'Street Light Lamp Dim',
                      'Flood Light Lamp Missing', 
                      'Wood Pole Knocked Down', 
                      'Wood Pole Leaning', 
                      'Wood Pole Missing']
lights = lights[lights['Descriptor'].isin(descriptors_useful)]

# Dates to DateTime

In [7]:
lights['CreatedDate'] = pd.to_datetime(lights['Created Date'], infer_datetime_format=True)
lights['ClosedDate'] = pd.to_datetime(lights['Closed Date'], infer_datetime_format=True)
collision['datetime'] = pd.to_datetime(collision['DATE'] + ' ' + collision['TIME'], infer_datetime_format=True)
lights = lights.drop(['Created Date', 'Closed Date'], 1)

# Merging Lights and Collision

In [39]:
bin_size = 0.06
binning = lambda x: round(x / bin_size) * bin_size
lights['lat_bin'] = lights['Latitude'].apply(binning)
lights['long_bin'] = lights['Longitude'].apply(binning)
collision['lat_bin'] = collision['LATITUDE'].apply(binning)
collision['long_bin'] = collision['LONGITUDE'].apply(binning)

In [40]:
lights['uniquekey'] = lights['Unique Key']
collision['uniquekey'] = collision['UNIQUE KEY']

In [41]:
#Make the db in memory
conn = sqlite3.connect(':memory:')
conn.text_factory = str

#write the tables
collision.to_sql('collision', conn, index=False)
lights.to_sql('lights', conn, index=False)

/home/lee/anaconda3/lib/python3.5/site-packages/pandas/core/generic.py:1201: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  chunksize=chunksize, dtype=dtype)


In [42]:
query = '''
        select
            collision.uniquekey, count(lights.uniquekey)
        from
            collision left join lights
                on datetime between CreatedDate and ClosedDate 
                and collision.lat_bin = lights.lat_bin
                and collision.long_bin = lights.long_bin
        group by
            collision.uniquekey
        '''

df = pd.read_sql_query(query, conn)

In [43]:
df.describe()

,uniquekey,count(lights.uniquekey)
count,7.928510e+05,792851.000000
mean,1.945638e+06,53.121420
std,1.528130e+06,40.028112
min,2.200000e+01,0.000000
25%,2.216295e+05,24.000000
50%,3.033589e+06,43.000000
75%,3.342113e+06,73.000000
max,3.631696e+06,215.000000


In [44]:
lights.Descriptor.unique()

array(['Street Light Out', 'Fixture/Luminaire Damaged', 'Lamppost Missing',
       'Lamppost Damaged', 'Street Light Lamp Missing',
       'Fixture/Luminaire Hanging', 'Fixture/Luminaire Missing',
       'Lamppost Knocked Down', 'Street Light Lamp Dim',
       'Flood Light Lamp Missing', 'Wood Pole Knocked Down',
       'Wood Pole Leaning', 'Wood Pole Missing'], dtype=object)

In [47]:
df['Lights Complaints  0.06'] = df['count(lights.uniquekey)']
df = df.drop('count(lights.uniquekey)', 1)

KeyError: 'count(lights.uniquekey)'

In [48]:
save['Lights Complaints 0.06'] = df['Lights Complaints 0.001']

In [ ]:
df['Lights Complaints 0.06'].describe()

In [50]:
save['UNIQUE KEY'] = save['uniquekey']
save.set_index(['UNIQUE KEY'], inplace=True)

In [52]:
save.describe()

,Lights Complaints 0.03,Lights Complaints 0.001,Lights Complaints 0.06
count,792851.000000,316240.000000,316240.000000
mean,13.516381,0.054272,52.942575
std,11.889359,0.270396,40.824616
min,0.000000,0.000000,0.000000
25%,4.000000,0.000000,24.000000
50%,11.000000,0.000000,41.000000
75%,20.000000,0.000000,72.000000
max,79.000000,12.000000,215.000000


In [51]:
save = save.drop('uniquekey', 1)

In [53]:
save.to_pickle('lights')